# Контентная рекомендательная система

Ты уже запрограммировал систему неперсональных рекомендаций, в этом занятии ты пройдешь все этапы разработки модели контентных рекомендаций, а значит перейдешь на следующий уровень кодинга. 

Какой у нас план?

 - Добавим необходимые библиотеки и загрузим датасеты «movies_metadata_fixed.csv», «credits.csv», «keywords.csv», «links_small.csv». 
 - Соберем в один датасет необходимые данные, добавим поля с актерами фильмов и ключевыми словами.
 - Обработаем данные, чтобы они были представлены в удобном виде и удалим ненужные элементы. Переведем описание фильмов в текстовый формат (в колонках представим набор слов).
 - Чтобы появилась возможность сравнивать описания фильмов, превратим текст в вектор. 
 - Найдем и сравним похожие фильмы по их **векторному представлению**. 

 Поехали!

Начни с импорта библиотек, подгружаем **Pandas** и **Ast** (с пакетом Literal_eval)

In [45]:
#импортируй библиотеки, которые мы использовали для решения первого задания
import warnings
import pandas as pd
from ast import literal_eval

warnings.filterwarnings('ignore')

На этот раз ты работаешь не только с таблицами, но и с текстами, а также с математическими объектами. Импортируем подходящие библиотеки и пакеты: **Scikit-learn** (пакеты CountVectorizer и cosine_similarity), **NLTK** (пакет SnowballStemmer).

In [46]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.snowball import SnowballStemmer

Загружаем датасет из файла **«movies_metadata_fixed.csv»**

In [47]:
#Загрузи файл movies_metadata_fixed.csv в переменную dataset
dataset = pd.read_csv('./the-movies-dataset/movies_metadata_fixed.csv')
dataset.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92


Аналогично первому занятию подготовь колонки **Жанры** и **Год выпуска фильма**

In [48]:
#Обработай жанры в датасете
literal_eval(dataset['genres'].iloc[0])
[i['name'] for i in literal_eval(dataset['genres'].iloc[0])]
dataset['genres'] = dataset['genres'].fillna('[]')\
                                     .apply(literal_eval)\
                                     .apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
dataset['genres'].head()

0     [Animation, Comedy, Family]
1    [Adventure, Fantasy, Family]
2               [Romance, Comedy]
3        [Comedy, Drama, Romance]
4                        [Comedy]
Name: genres, dtype: object

In [49]:
#Отдели год выпуска фильма от полной даты выхода
dataset['year'] = pd.to_datetime(dataset.release_date).dt.year

Кроме информации о фильмах нам нужны будут дополнительные данные, их мы загрузим из других файлов.

В файле **credits.csv** хранится информация о съемочной группе, а база **keywords.csv** содержит информацию о ключевых словах-  это слова-теги, которые характеризуют каждый из фильмов.

In [50]:
credits = pd.read_csv('./the-movies-dataset/credits.csv')
keywords = pd.read_csv('./the-movies-dataset/keywords.csv')

Чтобы прикрепить колонки с нужной информацией к рабочему датасету с фильмами из последних файлов, воспользуемся функцией **merge** из пакета **Pandas**. Объединяем колонки по столбцу "id". 

In [51]:
dataset = pd.merge(dataset, credits, on='id')
dataset = pd.merge(dataset, keywords, on='id')
dataset.shape

(46628, 28)

Чтобы ускорить вычисления и быстро создать прототип рекомендательной системы, создатели датасета подготовили выборку из 9219 фильмов **links_small.csv**. На этом наборе данных ты быстрее протестируешь идеи, а уже потом сможешь применить ко всей базе фильмов целиком. 

In [52]:
links_small = pd.read_csv('./the-movies-dataset/links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [53]:
smd = dataset[dataset['id'].isin(links_small)]
smd.shape

(9219, 28)

Как видишь, наш датасет заметно уменьшился, можем переходить к обработке данных.

___

# Подготовка данных

Перед тобой стоит задача сравнить фильмы между собой, чтобы подобрать наиболее похожие. Как ты будешь определять эту "похожесть"? 

Есть пара идей:
 - Фильмы мог снять один и тот же режиссер
 - В фильмах сыграли одни и те же актеры
 - Наконец, сюжеты фильмов совпадают или один фильм продолжает другой
 
Приведем в пример трилогию "Хоббит". Три части киноистории снял одним и тот же режиссер, актерский состав почти полностью совпадает, да и сюжет второй и третьей частей продолжают первую.

Как раз эту информацию ты и попроббуешь собрать из нескольких колонок, а затем превратить в текст. Сейчас мы используем только эти поля из базы данных для примера, но есть и другие параметры, по которым фильмы могут совпадать (по смыслу). Ты можешь самостоятельно поработать с ними, когда завершишь общую часть второго задания.
___

Начни с переменной **cast**. Этот формат ты уже использовал в первом задании, поэтому знаешь, как с ним справиться. Сначала примени функцию **literal_eval**.

In [54]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd.cast.head()

0    [{'cast_id': 14, 'character': 'Woody (voice)',...
1    [{'cast_id': 1, 'character': 'Alan Parrish', '...
2    [{'cast_id': 2, 'character': 'Max Goldman', 'c...
3    [{'cast_id': 1, 'character': 'Savannah 'Vannah...
4    [{'cast_id': 1, 'character': 'George Banks', '...
Name: cast, dtype: object

На выходе **literal_eval** получаем список словарей в виде объектов языка Python. Информация, которая нам нужна, лежит в полях **name**. Сформируй список актерского состава тем же способом, что и в первом задании

In [55]:
#Сформируй список актерского состава, используя лямбда-функцию
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd.cast.head()

0    [Tom Hanks, Tim Allen, Don Rickles, Jim Varney...
1    [Robin Williams, Jonathan Hyde, Kirsten Dunst,...
2    [Walter Matthau, Jack Lemmon, Ann-Margret, Sop...
3    [Whitney Houston, Angela Bassett, Loretta Devi...
4    [Steve Martin, Diane Keaton, Martin Short, Kim...
Name: cast, dtype: object

Отлично! Теперь для каждого фильма у тебя есть список актеров. Теперь это поле мы превратим в одну большую строку. Предлагаем учитывать только главные роли фильма, оставим первые пять имен актеров, а остальные опустим.

Чтобы отличать актеров с одинаковыми именами, но разными фамилиями, переведи строку в нижний регистр и убери пробелы в именах. Имя **Tom Hanks** запишется как **tomhanks**, тоже самое произойдет и с другими. Проделав эту операцию, алгоритм уже не будет считать фильмы похожими, просто потому что у актеров одинаковые имена.

In [56]:
#оставь только пять первых актеров
smd['cast'] = smd['cast'].apply(lambda cast: cast[:5] if len(cast) >= 5 else cast)

#переведи в нижний регистр имена и удали пробелы между именем и фамилией
smd['cast'] = smd['cast'].apply(lambda cast: [str.lower(actor.replace(" ", "")) for actor in cast])

#объедини актеров каждого фильма в одну строку, разделяя имена пробелами
smd['cast_str'] = smd['cast'].apply(lambda cast: ' '.join(cast))
smd.cast_str.head()

0    tomhanks timallen donrickles jimvarney wallace...
1    robinwilliams jonathanhyde kirstendunst bradle...
2    waltermatthau jacklemmon ann-margret sophialor...
3    whitneyhouston angelabassett lorettadevine lel...
4    stevemartin dianekeaton martinshort kimberlywi...
Name: cast_str, dtype: object

На очереди колонка **crew**. В ней хранится важная информация, а именно в поле **job** — здесь есть имена и должности всех людей, которые принимали участие в съемках фильма. Давай используем функцию **literal_eval** и рассмотрим поле в деталях.

In [57]:
#используй функцию literal_eval
smd['crew'] = smd['crew'].apply(literal_eval)
smd.crew.head()

0    [{'credit_id': '52fe4284c3a36847f8024f49', 'de...
1    [{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...
2    [{'credit_id': '52fe466a9251416c75077a89', 'de...
3    [{'credit_id': '52fe44779251416c91011acb', 'de...
4    [{'credit_id': '52fe44959251416c75039ed7', 'de...
Name: crew, dtype: object

In [58]:
smd['crew'].iloc[0][0]

{'credit_id': '52fe4284c3a36847f8024f49',
 'department': 'Directing',
 'gender': 2,
 'id': 7879,
 'job': 'Director',
 'name': 'John Lasseter',
 'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'}

Имя режиссера ты найдешь в поле **Director** (важно! это поле может отсутствовать). Напиши такую функцию, чтобы извлечь имя режиссера для каждого фильма, а если информации нет, то функция возвращает значение **np.nan**.

In [61]:
def get_director(crew):
    for member in crew:
        #проверьте, что должность - режиссер:
        if member['job'] == 'Director':
            #верните ответом имя режиссера
            return member['name']
    return np.nan
smd['director'] = smd['crew'].apply(get_director)
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd.director.head()

0      johnlasseter
1       joejohnston
2      howarddeutch
3    forestwhitaker
4      charlesshyer
Name: director, dtype: object

Последняя колонка, которая тебе нужна - это **keywords**, те самые ключевые слова-теги. Давай посмотрим, как она выглядит. 

In [62]:
smd['keywords'].head().apply(literal_eval).iloc[0]

[{'id': 931, 'name': 'jealousy'},
 {'id': 4290, 'name': 'toy'},
 {'id': 5202, 'name': 'boy'},
 {'id': 6054, 'name': 'friendship'},
 {'id': 9713, 'name': 'friends'},
 {'id': 9823, 'name': 'rivalry'},
 {'id': 165503, 'name': 'boy next door'},
 {'id': 170722, 'name': 'new toy'},
 {'id': 187065, 'name': 'toy comes to life'}]

Чтобы извлечь данные, обратимся к полю **name** в каждом элементе списка

In [63]:
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd.keywords.head()

0    [jealousy, toy, boy, friendship, friends, riva...
1    [board game, disappearance, based on children'...
2    [fishing, best friend, duringcreditsstinger, o...
3    [based on novel, interracial relationship, sin...
4    [baby, midlife crisis, confidence, aging, daug...
Name: keywords, dtype: object

Ты хочешь рекомендовать похожие фильмы, а значит есть смысл, оставить только те ключевые слова, которые встречаются у нескольких фильмов. Исключим ключевые слова, которые встречаются лишь раз во всем датасете:
- соберем все ключевые слова всех фильмов в один большой список
- посчитаем, сколько раз встречалось каждое ключевое слово
- удалим те ключевые слова, которые встретились только раз

In [64]:
s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1)\
       .stack()\
       .reset_index(level=1, drop=True)
s.name = 'keyword'
s = s.value_counts()
s = s[s > 1]

In [65]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd.keywords.head()

0    [jealousy, toy, boy, friendship, friends, riva...
1    [board game, disappearance, based on children'...
2         [fishing, best friend, duringcreditsstinger]
3    [based on novel, interracial relationship, sin...
4    [baby, midlife crisis, confidence, aging, daug...
Name: keywords, dtype: object

Ты собрал список слов-тегов для каждого фильма, но это еще не всё. Если вчитаться в список, то некоторые ключевые слова повторяются, хотя имеют разные формы (например, единственное и множественное число).

In [66]:
s['friend'], s['friends']

(6, 88)

In [67]:
s['boy'], s['boys']

(31, 3)

Представь, в фильме есть ключевые слова **friend и boy**, а в другом **friends и boys**. Алгоритм будет считать эти фильмы непохожими, а значит мы потеряем информацию.

Чтобы разобраться с этой проблемой, используй специальный алгоритм - **стеммер** (stemmer) от англ. слова **stem** - корень.  
Этот алгоритм поможет тебе выделить корень слова и превратить всех **friends** во **friend**, а **boys** в **boy**.

Посмотри, как это работает:

In [68]:
stemmer = SnowballStemmer('english')
stemmer.stem('dogs'), stemmer.stem('friends'), stemmer.stem('boys')

('dog', 'friend', 'boy')

Алгоритм сработал, как нужно: для твоей задачи вполне подходит. Давай обработаем каждое ключевое слово для каждого фильма этим алгоритмом. Это улучшит результат поиска похожих фильмов (скоро сам в этом убедишься). 

Удобнее всего такое преобразование выразить как отдельную функцию. Напиши функцию, чтобы на входе она принимала список слов, а на выходе возращала уже обработанный.

In [69]:
def stem_keywords(x):
    stemmed_tokens = []
    for token in x:
        try:
            new_token = stemmer.stem(token)
            stemmed_tokens.append(new_token)
        except:
            stemmed_tokens.append(token)
    return stemmed_tokens

#примените написанную выше функцию ко всему столбцу
smd['keywords'] = smd['keywords'].apply(lambda x: stem_keywords(x))
smd['keywords'] = smd['keywords'].apply(lambda x: [i.replace(" ", "").lower() for i in x])

In [70]:
smd['keywords_str'] = smd['keywords'].apply(lambda x: ' '.join([str(i) for i in x]))
smd.keywords_str.head()

0    jealousi toy boy friendship friend rivalri boy...
1    boardgam disappear basedonchildren'sbook newho...
2                   fish bestfriend duringcreditssting
3    basedonnovel interracialrelationship singlemot...
4    babi midlifecrisi confid age daughter motherda...
Name: keywords_str, dtype: object

____

# Векторизация текстов

Вау! Видим, что ты добрался практически до финиша разработки контентной системы рекомендаций. Все колонки, что ты так кропотливо готовил, пора направить в дело: keywords, director, cast, genres.

Нужно объединить эти колонки в одну большую строку для каждого фильма, так у каждого объекта будет подробное текстовое описание. Объединяй колонки через пробел (чтобы слова не слиплись) и переводи все в нижний регистр.

In [72]:
def concat_fields(data):
    concat = data['keywords'] + data['cast'] + [data['director']] + data['genres']
    result = ' '.join([str(i).lower() for i in concat])
    return result
smd['soup'] = smd.apply(lambda x: concat_fields(x), axis=1)

In [73]:
smd['title'].iloc[0], smd['soup'].iloc[0]

('Toy Story',
 'jealousi toy boy friendship friend rivalri boynextdoor newtoy toycomestolif tomhanks timallen donrickles jimvarney wallaceshawn johnlasseter animation comedy family')

Обрати внимание, как выглядит запись о  фильме "История игрушек". В строке есть информация и об актерах, и о режиссере, и о жанре фильма, и про ключевые слова не забыли. Если для человека эта строка абсолютно бесполезная, то компьютер с ней быстро справится.

___

Помнишь в задании мы рассмотрели векторное представление текста, чтобы рекомендации заработали? Начинаем работу по преобразованию данных. Сейчас мы превратим этот текст в математический объект - **вектор**. Как ты знаешь, вектор это направленный отрезок. Это довольно простой объект, но для нас он будет ключевым элементом в системе контентных рекомендаций. 

Когда мы переведем описание каждого фильма в вектор, то как и с любыми другими векторами, ты, наконец, сможешь автоматически сравнивать фильмы между собой. 

### CountVectorizer

Чтобы преобразовать текстовое описание в вектор нам будет нужен объект **CountVectorizer**. Он умеет превращать огромный массив текстов в набор векторов. Посмотри на пример:

In [74]:
vectorizer = CountVectorizer()
vectors = vectorizer.fit_transform(['quick brown fox jumped over lazy dog', 
                          'i like dog', 
                          'i like cat'])
pd.DataFrame(data=vectors.toarray(), 
             index=['Doc1', 'Doc2', 'Doc3'],
             columns=vectorizer.get_feature_names())

,brown,cat,dog,fox,jumped,lazy,like,over,quick
Doc1,1,0,1,1,1,1,0,1,1
Doc2,0,0,1,0,0,0,1,0,0
Doc3,0,1,0,0,0,0,1,0,0


На входе было 3 предложения:
- 'quick brown fox jumped over lazy dog', 
- 'i like dog',
- 'i like cat'

На выходе мы получили 3 вектора. Обрати внимание, как эти вектора выглядят. Координаты вектора - это слова, поэтому, если в предложении было слово **dog**, то в координате под названием **dog** будет число 1.

Точно так же, чуть позже, ты превратишь все описания фильмов в вектора. Но есть ещё один вопрос, как сравнивать эти вектора?

Простой вопрос, какое предложение больше похоже на **quick brown fox jumped over lazy dog**: 
    - i like dog
    - i like cat
Правильный ответ - i like dog. В исходном предложении есть слово dog, но нет слова cat.

Чтобы это понять математически, используют скалярное произведение векторов:

In [75]:
cosine_similarity(vectors)

array([[1.        , 0.26726124, 0.        ],
       [0.26726124, 1.        , 0.5       ],
       [0.        , 0.5       , 1.        ]])

Функция **cosine_similarity** вычислит скалярное произведение между всеми парами векторов. В примере у нас 3 вектора - 3 предложения, поэтому на выходе функции мы получим табличку размера **3x3**.  
В первой строке таблицы записаны значения скалярных произведений между первым предложением и остальными двумя. Легко заметить, что расстояние вектора до самого себя = 1 (т.к. предложение на 100% совпадает с самим собой), расстояние до второго предложения = 0.26, до третьего = 0. То есть благодаря рассчитаным значениям мы сделаем вывод, что второе предложение гораздо ближе к первому, чем третье.
___

А теперь давай провернём этот трюк с нашими фильмами Создай объект **CountVectorizer** и передай в функцию **fit_transform** все текстовые описания фильмов.

In [76]:
count = CountVectorizer(ngram_range=(1, 2), min_df=2)
count_matrix = count.fit_transform(smd['soup'])
count_matrix.shape

(9219, 21384)

Ты получил матрицу размером **9219x21346** - это значит, что все **9219** фильмов превратились в векторы размером **21346** элементов. 

Заметь, что объект CountVectorizer дает возможность установить дополнительные настройки
 - параметр **ngram_range=(1,2)** помогает учитывать не только отдельные слова, но и пары слов
 - параметр **min_df=2** отфильтрует все слова, которые встречались меньше чем в двух фильмах
 
Что нам даёт информация о размере матрицы? В нашем датасете **21346** уникальных слов и пар слов, которые встречались не менее чем в двух фильмах.

Теперь осталось вычислить скалярные произведения между всеми парами фильмов. Повтори все то же, что и в примере выше:

In [77]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim.shape

(9219, 9219)

В уменьшенном датасете было **9219** фильмов, поэтому матрица предсказаний имеет размеры **9219x9219**

# Функция рекомендации фильма

Ты уже сделал большую работу, теперь всё готово для написания функции рекомендации фильма. Тебе нужно написать функцию, которая принимает на вход фильм, а в ответ советует другие, похожие на него. Фильмы с самым минимальным расстоянием до фильма на входе в итоге и попадут в список рекомендаций.

In [78]:
smd = smd.reset_index()
# Сохрани в переменную title колонку с названиями фильмов из датасета smd
titles = smd.title
indices = pd.Series(smd.index, index=smd['title'])

In [79]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

### Ура, все готово! Проверим твою систему в работе

In [84]:
get_recommendations('The Hobbit: An Unexpected Journey').head(10)

3899    The Lord of the Rings: The Fellowship of the Ring
8833            The Hobbit: The Battle of the Five Armies
4436                The Lord of the Rings: The Two Towers
8537                  The Hobbit: The Desolation of Smaug
5074        The Lord of the Rings: The Return of the King
1693                                The Lord of the Rings
8867                                             Warcraft
477                                            The Shadow
5852                                           The Hobbit
2730                      Baby: Secret of the Lost Legend
Name: title, dtype: object

Посмотри, самые похожие фильмы на кино "The Hobbit: An Unexpected Journey" тоже будут фильмами про хоббитов. Система использует описания фильмов, чтобы выдавать рекомендации — ты отлично справился со вторым заданием. Остается сделать последний шаг в финальном проекте, разработать систему коллаборативной фильтрации. Об этом мы поговорим в третьем задании. 

In [85]:
get_recommendations('The Terminator').head(15)

522                Terminator 2: Judgment Day
4739       Terminator 3: Rise of the Machines
8854                       Terminator Genisys
974                                    Aliens
5924                                 Fortress
7296                     Terminator Salvation
6394                             District B13
6967                                 Doomsday
2412                                  RoboCop
5296                                   Zardoz
5638    Sky Captain and the World of Tomorrow
7403                                    Gamer
7502                          The Book of Eli
8042                         The Darkest Hour
4742    The League of Extraordinary Gentlemen
Name: title, dtype: object